In [ ]:
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
import fsl_mrs.utils.mrs_io as mrs_io
from fsl_mrs.utils.preproc import nifti_mrs_proc as proc

In [ ]:
from pathlib import Path

# Path to the directory 
NOTEBOOK_DIR = Path().resolve()
print(NOTEBOOK_DIR)

part = '004'
sup_data  = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_correction/raw_mPress/{part}/sup_raw.nii.gz')
unsup_data = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_correction/raw_mPress/{part}/unsup_raw.nii.gz')

#Note: the water unsuppressed data is used as reference for coil combintation and eddy current correction
#By default it is identical to the normal water unsupprssed reference, just with more repeats
avg_ref_data = proc.average(unsup_data, 'DIM_DYN')



In [ ]:
#Preprocess the water suppressed data


N1=4
N=4
supp_data = proc.coilcombine(sup_data, reference=avg_ref_data)
eccc_data = proc.coilcombine(avg_ref_data, reference=avg_ref_data)

supp_data1=proc.apply_fixed_phase(supp_data, 180)
eccc_data1=proc.apply_fixed_phase(eccc_data, 180)

supp_data2 = proc.align(supp_data1, 'DIM_DYN', ppmlim=(1.8,4.2))
supp_data3 = proc.average(supp_data2, 'DIM_DYN')


supp_data5 = proc.truncate_or_pad(supp_data3, -N1, 'first')

limits = [-0.15,0.15]
limunits = 'ppm'

supp_data6 = proc.remove_peaks(supp_data5, limits, limit_units=limunits)
supp_data7 = proc.shift_to_reference(supp_data6, 2.01, (1.95, 2.07))

final_sup = proc.apply_fixed_phase(proc.phase_correct(supp_data7, (1.9, 2.1)), 180)
final_sup2 = proc.align(final_sup, 'DIM_EDIT', ppmlim=(2.8, 4), figure=False )

In [ ]:
#Perform Sepctral editing as a sanity check

diff = (proc.subtract(proc.align(final_sup, 'DIM_EDIT', ppmlim=(2.8, 4), figure=False ),dim='DIM_EDIT', figure=False))

diff = proc.shift_to_reference(diff, 2.01, (1.9,2.1))
diff= proc.apply_fixed_phase(proc.phase_correct(diff, (1.9,2.1)), 180)
plt.figure()
_=diff.plot()

In [ ]:
#Preprocess unsuppressed data

unsupp_data = proc.coilcombine(unsup_data, reference=avg_ref_data)
unsupp_data1=proc.apply_fixed_phase(unsupp_data, 180)
unsupp_data2 = proc.align(unsupp_data1, 'DIM_DYN', ppmlim=(0,9), figure=False )
unsupp_data3 = proc.average(unsupp_data2, 'DIM_DYN')
unsupp_data4 = proc.truncate_or_pad(unsupp_data3, -N, 'first')

In [ ]:
#Plot both spectral edits as a sanity check 
diff2 = proc.apply_fixed_phase(proc.phase_correct(proc.shift_to_reference(proc.remove_peaks(proc.apply_fixed_phase(proc.subtract(proc.align(proc.phase_correct(unsupp_data4,ppmlim=(1.8,9)), 'DIM_EDIT', ppmlim=(1.8,9), figure=False ),dim='DIM_EDIT', figure=False), 180), limits, limit_units=limunits),2.01, (1.9, 2.1)), (1.9, 2.1)), 180)

plt.figure()
_=diff.plot()
_=diff2.plot()

In [ ]:
final_sup2.save(rf'{NOTEBOOK_DIR}/Data_for_correction/mPress_preproc/{part}/sup_preproc.nii.gz')
unsupp_data4.save(rf'{NOTEBOOK_DIR}/Data_for_correction/mPress_preproc/{part}/unsup_preproc.nii.gz')
